In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import pandas as pd

# Get Episode/Season Transcripts

In [15]:
def buildUrl(e):
    e = e.split(' ')
    e = '_'.join(e)
    e = e.rstrip("\n\r")
    url = 'http://adventuretime.wikia.com/wiki/'+e
    return url

def getBsObj(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read(), "lxml")
    except AttributeError as e:
        return None
    return bsObj

def getQuotes(url):
    bsObj = getBsObj(url)
    quotes = []
    try:
        for b in bsObj.findAll("dd"):
            quotes.append(b.get_text())
    except AttributeError as e:
        return None
    return quotes

def cleanQuotes(url):
    epi = urlClean(url)
    quotes = getQuotes(url)
    list_ = []
    try:
        for quo in quotes:
            quote = quo.split(':')
            if len(quote)>1:
                row = [quote[0].strip(),quote[1].strip(), epi]
                list_.append(row)
    except TypeError as t:
        return None
    quotes = pd.DataFrame(list_, columns = ['character', 'text', 'episode'])
    return quotes

def urlClean(url):
    # takes url and returns episode
    # sloppy coding... for now... 
    epi = url.replace('http://adventuretime.wikia.com/wiki/','')
    epi = epi.replace('/Transcript','')
    epi = epi.replace('_',' ')
    return epi


In [38]:
file = 's01_/season_01.txt'

with open(file, 'r') as e:
    episodes = e.readlines()
    
urls = [buildUrl(x) for x in episodes]
transcripts = pd.DataFrame(columns = ['character', 'text', 'episode'])

for u in urls:
    episode = cleanQuotes(u)
    transcripts = transcripts.append(episode)
    transcripts = transcripts.reset_index(drop=True)

maybe... type in episode... does string to see if each word in episode name is in url, then goes to that url??? idk... everything is fucking complicated.

sloppy coding but this version has everycharacter aligned with a line and an episode... next step is to clean up the subtitles!!!!!!!!!

# Get Episode/Season Subtitles

In [30]:
def getSRT(srt):
    with open(srt, "r") as srt:
        array = []
        for line in srt:
            if len(line) > 4:
                line = line.rstrip('r\n')
                line = line.strip('\ufeff')
                array.append(line)
    return array

def cleanSRT(srt):
    lists = []
    episode = srtClean(srt)
    srt = getSRT(srt)
    for m in srt:
        t = len(lists)
        if m.startswith('00:'):
            m = m.split(' --> ')
            start = m[0].replace(',',':')
            end = m[1].replace(',',':')
            lists.append([start,end])
        else:
            m = m.replace('- ', '')
            m = m.replace('--', '')
            m = m.replace('</i>','')
            m = m.replace('<i>','')
            lists[t-1].append(m)

    df = pd.DataFrame(lists, columns=['start','end','c','d'])
    df = df.fillna('')
    df['text'] = df[['c', 'd']].apply(lambda x: ' '.join(x), axis=1)
    df = df[['start','end','text']]
    df['text'] = df['text'].apply(lambda x: x.strip())
    df['episode'] = episode
    return df

def srtClean(srt):
    # sloppy coding... for now... 
    srt = srt.replace('s01_/','')
    srt = srt.replace('.srt','')
    return srt

In [413]:
subtitles = pd.DataFrame(columns = ['start', 'end','text', 'episode'])
import glob
for file in glob.glob("s01_/*.srt"):
    episode = cleanSRT(file)
    subtitles = subtitles.append(episode)
    subtitles = subtitles.reset_index(drop=True)

by comparing the subtitles verse transcripts... looks like i should clean the subtitles from any non talking text such as... [GASP] and EVIL VOICE: and ♪ Adventure Time ♪.

In [414]:
def cleanSubText(x):
    import re
    # ASSUME CAPS PHRASES START LINES!
    # removes stuff like 'ALL:'
    x = x.split(':',1)
    if len(x) > 1:
        x = x[1]
    else:
        x = x[0]
    # removes ()s and []s and ♪♪s
    x = re.sub("[\(\[\♪].*?[\)\]\♪]", "", x)
    x = x.strip()
    return x


subtitles['text'] = subtitles['text'].map(cleanSubText)
subtitles = subtitles[subtitles['text'] != '']
subtitles = subtitles.reset_index(drop=True)
subtitles = subtitles.iloc[::-1]
subtitles = subtitles.reset_index(drop=True)

#drp = []
# lloks ate ach sentence to see if it is lowercased
# if lowercased assumes its appart of next sentence
# adds itself to next sentence
# repeats for current row

def buildSents(df):
    drop = []
    for i in range(df.shape[0]):
        try:
            if df.iat[i, 2][0].islower():
                df.at[i+1, 'text'] = df.iat[i+1, 2] + ' ' + df.iat[i, 2]
                df.at[i+1, 'end'] = df.iat[i, 1]
                drop.append(i)
            else:
                pass
        except:
            pass


    df.drop(drop, inplace=True)
    df = df.reset_index(drop=True)
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    return df

subtitles = buildSents(subtitles)

each subtitle needs to be a sentence!!!!! once i finish this. then I can try to match the sentences with the transcritps. ill prolly have to breake the transcripts inten sentences as well!

# Map Characters to Times!!
In this section im going to use the order of the trans and subs files to determine the character at everytime stamp. Ill initially take subtext and see if each word is in transtext. if yes, ill take subtext then check if the next subtext is located in the same trans text. if not ill move forward. i shouldnt be iterating over every line each in file for each line in the other file... ordering should help with most problems. it might be easier to break the transfiles into sentences... but maybe not. lets do that after. 

In [415]:
subs = subtitles[subtitles['episode'] == 'Wizard (episode)']
subs = subs.reset_index(drop=True)
subs['character'] = ''
trans = transcripts[transcripts['episode'] == 'Wizard (episode)']
trans = trans.reset_index(drop=True)

In [416]:
def sen2list(x):
    x = x.lower()
    x = re.sub('[^A-Za-z0-9]+', ' ', x)
    x = x.split()
    return x

def wordMatcher(trans, subs):
    count = sum(w in trans for w in subs)
    length = len(subs)

    if count == 0:
        # no words were found
        #print('Terrible match!')
        return False
    elif length == count:
        # all words were found
        #print('Perfect match!')
        return True
    elif length - count < 2:
        # one word wasn't found
        #print('Decent match!')
        return True
    else:
        # more than one word wasn't found
        #print('Terrible match!')
        return False
from fuzzywuzzy import fuzz
   
def wordMatcher2(trans, subs):
    from fuzzywuzzy import fuzz
    if fuzz.partial_ratio(trans, subs) > 85:
        return True
    else:
        return False

In [435]:
"""
I need to have a try three trans and if fail... jump ship and come back to that index with the next subs

looks like when there isn't a complete match such as this... 
then everything jumps ship...

 Means he's evil, I guess. 
 It means he's evil, I guess.
 
 Means he's evil, I guess. 
 Well, evil, sure, but mostly, he's unattractive, and unattractive people are desperate.
 
 Hey, kid... You want some magic powers...? Magic powers for absolutely free... 
 Hey, kid, you want some magic powers?
 
 Alright... Free magical powers and... [Pulling left arm off] Eh! And this keychain. 
 All right.
 
  [One tadpole is lagging behind this time] So... you wish to be inducted into the ancient order of—[Only one tadpole speaking] Dang it, Leonard!! If we don't talk at the same time, nobody can understand us! 
 ...ancient order
"""
j = 0
i = 0
while i < len(subs):
    try:
        # if true update i
        if wordMatcher2(trans.iat[j,1],subs.iat[i,2]):
            print('ooo\n',trans.iat[j,1],'\n',subs.iat[i,2])
            subs.at[i,'character'] = trans.iat[j,0]
            i += 1
       # if false update j
        elif wordMatcher2(trans.iat[j+1,1],subs.iat[i,2]):
            print('booo\n',trans.iat[j,1],'\n',subs.iat[i,2])
            j += 1
        elif wordMatcher2(trans.iat[j+2,1],subs.iat[i,2]):
            print('booo\n',trans.iat[j,1],'\n',subs.iat[i,2])
            j += 1
        else:           
            print('booo\n',trans.iat[j,1],'\n',subs.iat[i,2])
            i += 1
    except:
        print('Checked all phrases?!?')
        break


ooo
 Hey, Jake. 
 Hey, Jake?
booo
 Hey, Jake. 
 Yeah?
ooo
 Yeah? 
 Yeah?
booo
 Yeah? 
 You ever wonder what a demon's heart looks like?
ooo
 You ever wonder what a demon's heart looks like? 
 You ever wonder what a demon's heart looks like?
booo
 You ever wonder what a demon's heart looks like? 
 Of course, dude.
ooo
 Of course, dude. 
 Of course, dude.
booo
 Of course, dude. 
 You ever wonder what a demon's heart tastes like?
ooo
 You ever wonder what a demon's heart tastes like? 
 You ever wonder what a demon's heart tastes like?
booo
 You ever wonder what a demon's heart tastes like? 
 Hey, kid, you want some magic powers?
ooo
 Hey, kid... You want some magic powers...? Magic powers for absolutely free... 
 Hey, kid, you want some magic powers?
ooo
 Hey, kid... You want some magic powers...? Magic powers for absolutely free... 
 Magic powers for absolutely free.
booo
 Hey, kid... You want some magic powers...? Magic powers for absolutely free... 
 Heck yeah!
ooo
 Heck yeah! 
 Heck y

booo
 [One tadpole is lagging behind this time] So... you wish to be inducted into the ancient order of—[Only one tadpole speaking] Dang it, Leonard!! If we don't talk at the same time, nobody can understand us! 
 Thanks to us, the village is saved, and we reclaimed a few of your twilight years.
booo
 [One tadpole is lagging behind this time] So... you wish to be inducted into the ancient order of—[Only one tadpole speaking] Dang it, Leonard!! If we don't talk at the same time, nobody can understand us! 
 Don't forget to thank me, for this was my plan fo all of these 847 years.
booo
 [One tadpole is lagging behind this time] So... you wish to be inducted into the ancient order of—[Only one tadpole speaking] Dang it, Leonard!! If we don't talk at the same time, nobody can understand us! 
 No way.
booo
 [One tadpole is lagging behind this time] So... you wish to be inducted into the ancient order of—[Only one tadpole speaking] Dang it, Leonard!! If we don't talk at the same time, nobody 

In [433]:
trans['clean'] = trans['text'].map(cleanSubText)
trans[trans['character'] == 'Finn']

In [ ]:
# clean trans
# break trans into sentences
# make list of character phrases
# search trans in subs

In [443]:
def transSplit(x):
    x = x.lower()
    x = re.split("[!?.,]", x)
    x = [s for s in x if len(s) > 1]
    return x

trans['clean'] = trans['text'].map(cleanSubText)
trans['clean'] = trans['clean'].map(transSplit)
trans.to_csv('trans.csv')

In [449]:
trans.iat[2,3][0].split()

['you', 'ever', 'wonder', 'what', 'a', 'demon']